In [ ]:
import requests
import json

# Tạo session để duy trì trạng thái đăng nhập
session = requests.Session()

# 1️⃣ Lấy CSRF token
session.get("https://quanly.nhatrovn.vn/login")
csrf_token = session.cookies.get("XSRF-TOKEN", "")

# 2️⃣ Đăng nhập
login_response = session.post(
    "https://quanly.nhatrovn.vn/authenticate",
    headers={"X-CSRF-TOKEN": csrf_token},
    data={"_csrf": csrf_token, "j_username": "0866956654", "j_password": "12345678@X"}
)

if login_response.ok and "SUCCESS" in login_response.text:
    print("✅ Đăng nhập thành công!")

    all_rows = []  # Danh sách để lưu toàn bộ dữ liệu
    last_key = 0   # Giá trị `_lastKey` ban đầu

    while True:
        # 3️⃣ Gọi API tìm kiếm
        search_response = session.post(
            "https://quanly.nhatrovn.vn/main/room-sale/search",
            headers={"X-CSRF-TOKEN": csrf_token, "Referer": "https://quanly.nhatrovn.vn/main/room-sale/init"},
            data={"_lastKey": str(last_key), "sort-by": "1"}
        )

        if not search_response.ok:
            print("❌ Lỗi khi tìm kiếm:", search_response.status_code)
            break

        response_json = search_response.json()
        rows = response_json.get("rows", [])

        if not rows:  # Nếu rows rỗng, dừng vòng lặp
            print("✅ Không còn dữ liệu, kết thúc quá trình tìm kiếm.")
            break

        all_rows.extend(rows)  # Thêm dữ liệu vào danh sách tổng hợp
        last_key += 10  # Tăng giá trị _lastKey

        print(f"🔹 Đã lấy {len(rows)} dòng, tiếp tục với _lastKey = {last_key}...")

    # 4️⃣ Xuất dữ liệu ra file JSON
    with open("merged_results.json", "w", encoding="utf-8") as f:
        json.dump(all_rows, f, ensure_ascii=False, indent=4)

    print(f"✅ Dữ liệu đã được lưu vào merged_results.json với tổng {len(all_rows)} dòng!")
else:
    print("❌ Đăng nhập thất bại!", login_response.status_code, login_response.text)


✅ Đăng nhập thành công!
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 10...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 20...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 30...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 40...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 50...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 60...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 70...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 80...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 90...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 100...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 110...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 120...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 130...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 140...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 150...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 160...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 170...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 180...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 190...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 200...
🔹 Đã 

In [ ]:
import requests
import json
import time

# Danh sách districts
districts = [
    {"code": "785", "name_with_type": "Huyện Bình Chánh"},
    {"code": "787", "name_with_type": "Huyện Cần Giờ"},
    {"code": "783", "name_with_type": "Huyện Củ Chi"},
    {"code": "784", "name_with_type": "Huyện Hóc Môn"},
    {"code": "786", "name_with_type": "Huyện Nhà Bè"},
    {"code": "760", "name_with_type": "Quận 1"},
    {"code": "771", "name_with_type": "Quận 10"},
    {"code": "772", "name_with_type": "Quận 11"},
    {"code": "761", "name_with_type": "Quận 12"},
    {"code": "779", "name_with_type": "Quận 2"},
    {"code": "770", "name_with_type": "Quận 3"},
    {"code": "773", "name_with_type": "Quận 4"},
    {"code": "774", "name_with_type": "Quận 5"},
    {"code": "775", "name_with_type": "Quận 6"},
    {"code": "778", "name_with_type": "Quận 7"},
    {"code": "776", "name_with_type": "Quận 8"},
    {"code": "780", "name_with_type": "Quận 9"},
    {"code": "765", "name_with_type": "Quận Bình Thạnh"},
    {"code": "777", "name_with_type": "Quận Bình Tân"},
    {"code": "764", "name_with_type": "Quận Gò Vấp"},
    {"code": "768", "name_with_type": "Quận Phú Nhuận"},
    {"code": "766", "name_with_type": "Quận Tân Bình"},
    {"code": "767", "name_with_type": "Quận Tân Phú"},
    {"code": "769", "name_with_type": "Thành phố Thủ Đức"}
]

# Tạo session để duy trì trạng thái đăng nhập
session = requests.Session()

# 1️⃣ Lấy CSRF token
session.get("https://quanly.nhatrovn.vn/login")
csrf_token = session.cookies.get("XSRF-TOKEN", "")

# 2️⃣ Đăng nhập
login_response = session.post(
    "https://quanly.nhatrovn.vn/authenticate",
    headers={"X-CSRF-TOKEN": csrf_token},
    data={"_csrf": csrf_token, "j_username": "0866956654", "j_password": "12345678@X"}
)

# Kiểm tra đăng nhập thành công
if not (login_response.ok and "SUCCESS" in login_response.text):
    print("❌ Đăng nhập thất bại!")
    exit()

print("✅ Đăng nhập thành công!")

# URL API
wards_url = "https://quanly.nhatrovn.vn/common/get-wards"

# Dictionary chứa kết quả wards theo district-code
wards_data = {}

# Headers chung
headers = {
    "X-CSRF-TOKEN": csrf_token,
    "Referer": "https://quanly.nhatrovn.vn/main/room-sale/init"
}

# Gọi API cho từng district-code
for district in districts:
    district_code = district["code"]

    try:
        response = session.post(wards_url, headers=headers, data={"district-code": district_code})

        if response.status_code == 200:
            try:
                data = response.json()
                if isinstance(data, list):  # Đảm bảo dữ liệu trả về là danh sách JSON hợp lệ
                    wards_data[district_code] = data
                    print(f"✅ Đã lấy {len(data)} phường/xã cho district {district_code}")
                else:
                    print(f"⚠️ API trả về dữ liệu không hợp lệ cho district {district_code}: {data}")
            except json.JSONDecodeError:
                print(f"❌ Lỗi JSONDecodeError: Không thể parse JSON cho district {district_code}")
                print(f"🔹 Response text: {response.text}")
        else:
            print(f"❌ Lỗi HTTP {response.status_code} khi lấy dữ liệu district {district_code}")

    except requests.RequestException as e:
        print(f"❌ Request failed for district {district_code}: {e}")

    time.sleep(1)  # Tránh gửi request quá nhanh

# Lưu danh sách districts
with open("districts.json", "w", encoding="utf-8") as f:
    json.dump(districts, f, ensure_ascii=False, indent=4)

# Lưu danh sách wards
with open("wards.json", "w", encoding="utf-8") as f:
    json.dump(wards_data, f, ensure_ascii=False, indent=4)

print("✅ Dữ liệu đã được lưu vào districts.json và wards.json!")


✅ Đăng nhập thành công!
✅ Đã lấy 16 phường/xã cho district 785
✅ Đã lấy 7 phường/xã cho district 787
✅ Đã lấy 21 phường/xã cho district 783
✅ Đã lấy 12 phường/xã cho district 784
✅ Đã lấy 7 phường/xã cho district 786
✅ Đã lấy 10 phường/xã cho district 760
✅ Đã lấy 11 phường/xã cho district 771
✅ Đã lấy 10 phường/xã cho district 772
✅ Đã lấy 11 phường/xã cho district 761
✅ Đã lấy 11 phường/xã cho district 779
✅ Đã lấy 13 phường/xã cho district 770
✅ Đã lấy 12 phường/xã cho district 773
✅ Đã lấy 10 phường/xã cho district 774
✅ Đã lấy 11 phường/xã cho district 775
✅ Đã lấy 10 phường/xã cho district 778
✅ Đã lấy 10 phường/xã cho district 776
✅ Đã lấy 2 phường/xã cho district 780
✅ Đã lấy 16 phường/xã cho district 765
✅ Đã lấy 10 phường/xã cho district 777
✅ Đã lấy 13 phường/xã cho district 764
✅ Đã lấy 11 phường/xã cho district 768
✅ Đã lấy 15 phường/xã cho district 766
✅ Đã lấy 11 phường/xã cho district 767
✅ Đã lấy 34 phường/xã cho district 769
✅ Dữ liệu đã được lưu vào districts.json và

In [ ]:
import requests
import json
import time
from datetime import datetime

# Load dữ liệu districts và wards từ file
with open("districts.json", "r", encoding="utf-8") as f:
    districts = json.load(f)

with open("wards.json", "r", encoding="utf-8") as f:
    wards_data = json.load(f)

# Map tên quận/huyện sang mã code
district_map = {d["name_with_type"]: d["code"] for d in districts}

# Map tên phường/xã sang mã code
ward_map = {}
for district_code, ward_list in wards_data.items():
    for ward in ward_list:
        ward_map[ward["name_with_type"]] = ward["code"]

# Đọc dữ liệu cũ từ merged_results.json (nếu có)
try:
    with open("merged_results.json", "r", encoding="utf-8") as f:
        old_data = json.load(f)
except FileNotFoundError:
    old_data = []

# Chuyển danh sách cũ thành dictionary {_id: record} để tra cứu nhanh
old_data_dict = {str(record.get("_id")): record for record in old_data}

# Tạo session để duy trì trạng thái đăng nhập
session = requests.Session()

# 1️⃣ Lấy CSRF token
session.get("https://quanly.nhatrovn.vn/login")
csrf_token = session.cookies.get("XSRF-TOKEN", "")

# 2️⃣ Đăng nhập
login_response = session.post(
    "https://quanly.nhatrovn.vn/authenticate",
    headers={"X-CSRF-TOKEN": csrf_token},
    data={"_csrf": csrf_token, "j_username": "0866956654", "j_password": "12345678@X"}
)

if login_response.ok and "SUCCESS" in login_response.text:
    print("✅ Đăng nhập thành công!")

    all_rows = {}  # Dictionary để lưu dữ liệu mới với key là _id
    new_ids = []  # Danh sách lưu ID mới xuất hiện
    restored_ids = []  # Danh sách ID được khôi phục từ soft delete
    last_key = 0  # Giá trị `_lastKey` ban đầu

    while True:
        # 3️⃣ Gọi API tìm kiếm
        search_response = session.post(
            "https://quanly.nhatrovn.vn/main/room-sale/search",
            headers={"X-CSRF-TOKEN": csrf_token, "Referer": "https://quanly.nhatrovn.vn/main/room-sale/init"},
            data={"_lastKey": str(last_key), "sort-by": "1"}
        )

        if not search_response.ok:
            print("❌ Lỗi khi tìm kiếm:", search_response.status_code)
            break

        response_json = search_response.json()
        rows = response_json.get("rows", [])

        if not rows:  # Nếu rows rỗng, dừng vòng lặp
            print("✅ Không còn dữ liệu, kết thúc quá trình tìm kiếm.")
            break

        # Xử lý dữ liệu từng dòng
        for row in rows:
            row["_id"] = str(row.get("_id"))  # ID phải chuyển về string để tránh lỗi
            house_address = row.get("house_address", "")

            row["province_id"] = 79  # Thành phố Hồ Chí Minh luôn có province_id = 79
            row["district_id"] = None
            row["ward_id"] = None
            row["deleted_at"] = None  # Đánh dấu chưa bị xóa

            # Tách địa chỉ
            address_parts = house_address.split(", ")
            if len(address_parts) >= 3:
                ward_name = address_parts[-3].strip()  # Lấy phường
                district_name = address_parts[-2].strip()  # Lấy quận/huyện

                row["district_id"] = district_map.get(district_name, None)
                row["ward_id"] = ward_map.get(ward_name, None)

            # Nếu _id này chưa từng có trước đó → Thêm vào danh sách ID mới
            if row["_id"] not in old_data_dict:
                new_ids.append(row["_id"])

            # Nếu _id này từng bị đánh dấu xóa, khôi phục nó
            if row["_id"] in old_data_dict and old_data_dict[row["_id"]].get("deleted_at"):
                row["deleted_at"] = None  # Xóa trạng thái deleted_at
                restored_ids.append(row["_id"])  # Ghi nhận ID được khôi phục

            all_rows[row["_id"]] = row

        last_key += 10  # Tăng giá trị _lastKey
        print(f"🔹 Đã lấy {len(rows)} dòng, tiếp tục với _lastKey = {last_key}...")
        time.sleep(1)  # Tránh bị chặn

    # 4️⃣ Kiểm tra xem có bản ghi nào bị thiếu trong dữ liệu mới không
    now_str = datetime.now().isoformat()
    for old_id, old_record in old_data_dict.items():
        if old_id not in all_rows:  # Nếu _id này không còn trong dữ liệu mới
            old_record["deleted_at"] = now_str  # Đánh dấu soft delete
            all_rows[old_id] = old_record

    # 5️⃣ Xuất dữ liệu ra file JSON
    with open("merged_results.json", "w", encoding="utf-8") as f:
        json.dump(list(all_rows.values()), f, ensure_ascii=False, indent=4)

    print(f"✅ Dữ liệu đã được lưu vào merged_results.json với tổng {len(all_rows)} dòng!")

    # 6️⃣ Thông báo các ID mới xuất hiện
    if new_ids:
        print(f"🚀 Có {len(new_ids)} ID mới xuất hiện:")
        print("\n".join(new_ids))
    else:
        print("🔹 Không có ID mới nào.")

    # 7️⃣ Thông báo các ID được khôi phục từ soft delete
    if restored_ids:
        print(f"🔄 Có {len(restored_ids)} ID được khôi phục từ soft delete:")
        print("\n".join(restored_ids))
    else:
        print("🔹 Không có ID nào được khôi phục.")

else:
    print("❌ Đăng nhập thất bại!", login_response.status_code, login_response.text)


✅ Đăng nhập thành công!
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 10...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 20...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 30...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 40...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 50...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 60...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 70...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 80...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 90...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 100...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 110...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 120...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 130...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 140...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 150...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 160...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 170...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 180...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 190...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 200...
🔹 Đã 

In [ ]:
import requests
import json
import time
from datetime import datetime
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load dữ liệu districts và wards từ file
with open("districts.json", "r", encoding="utf-8") as f:
    districts = json.load(f)

with open("wards.json", "r", encoding="utf-8") as f:
    wards_data = json.load(f)

# Map tên quận/huyện sang mã code
district_map = {d["name_with_type"]: d["code"] for d in districts}

# Map tên phường/xã sang mã code
ward_map = {}
for district_code, ward_list in wards_data.items():
    for ward in ward_list:
        ward_map[ward["name_with_type"]] = ward["code"]

# Đọc dữ liệu cũ từ merged_results.json (nếu có)
try:
    with open("merged_results.json", "r", encoding="utf-8") as f:
        old_data = json.load(f)
except FileNotFoundError:
    old_data = []

# Chuyển danh sách cũ thành dictionary {_id: record} để tra cứu nhanh
old_data_dict = {str(record.get("_id")): record for record in old_data}

# Tạo session để duy trì trạng thái đăng nhập
session = requests.Session()

# 1️⃣ Lấy CSRF token
session.get("https://quanly.nhatrovn.vn/login")
csrf_token = session.cookies.get("XSRF-TOKEN", "")

# 2️⃣ Đăng nhập
login_response = session.post(
    "https://quanly.nhatrovn.vn/authenticate",
    headers={"X-CSRF-TOKEN": csrf_token},
    data={"_csrf": csrf_token, "j_username": "0866956654", "j_password": "12345678@X"}
)

if login_response.ok and "SUCCESS" in login_response.text:
    print("✅ Đăng nhập thành công!")

    all_rows = {}
    new_ids = []
    restored_ids = []
    last_key = 0

    def get_room_images(house_key, room_id):
        """Gọi API để lấy danh sách hình ảnh của phòng."""
        images_url = "https://quanly.nhatrovn.vn/main/room-sale/init-edit-images-room"

        try:
            response = session.post(
                images_url,
                headers={"X-CSRF-TOKEN": csrf_token},
                data={"house_key": house_key, "_id": room_id},
                timeout=5  # Giới hạn thời gian chờ 5s
            )
            if not response.ok:
                print(f"❌ Lỗi khi lấy hình ảnh phòng {room_id}")
                return []

            soup = BeautifulSoup(response.text, "html.parser")
            return [img["src"] for img in soup.find_all("img", {"class": "img-fluid"}) if "src" in img.attrs]
        except requests.exceptions.RequestException:
            print(f"⏳ Lỗi timeout khi lấy ảnh phòng {room_id}")
            return []

    while True:
        # 3️⃣ Gọi API tìm kiếm
        search_response = session.post(
            "https://quanly.nhatrovn.vn/main/room-sale/search",
            headers={"X-CSRF-TOKEN": csrf_token},
            data={"_lastKey": str(last_key), "sort-by": "1"}
        )

        if not search_response.ok:
            print("❌ Lỗi khi tìm kiếm:", search_response.status_code)
            break

        response_json = search_response.json()
        rows = response_json.get("rows", [])

        if not rows:
            print("✅ Không còn dữ liệu, kết thúc quá trình tìm kiếm.")
            break

        # Lưu danh sách tasks để chạy song song
        tasks = []
        with ThreadPoolExecutor(max_workers=10) as executor:
            for row in rows:
                row["_id"] = str(row.get("_id"))
                house_key = row.get("house_key", "")
                house_address = row.get("house_address", "")

                row["province_id"] = 79
                row["district_id"] = None
                row["ward_id"] = None
                row["deleted_at"] = None

                # Tách địa chỉ
                address_parts = house_address.split(", ")
                if len(address_parts) >= 3:
                    ward_name = address_parts[-3].strip()
                    district_name = address_parts[-2].strip()
                    row["district_id"] = district_map.get(district_name, None)
                    row["ward_id"] = ward_map.get(ward_name, None)

                # Gửi request lấy ảnh song song
                tasks.append(executor.submit(get_room_images, house_key, row["_id"]))

                all_rows[row["_id"]] = row

            # Nhận kết quả từ các requests chạy song song
            for future, row in zip(as_completed(tasks), rows):
                room_images = future.result()
                all_rows[row["_id"]]["images"] = room_images

                if row["_id"] not in old_data_dict:
                    new_ids.append(row["_id"])
                if row["_id"] in old_data_dict and old_data_dict[row["_id"]].get("deleted_at"):
                    row["deleted_at"] = None
                    restored_ids.append(row["_id"])

        last_key += 10
        print(f"🔹 Đã lấy {len(rows)} dòng, tiếp tục với _lastKey = {last_key}...")
        time.sleep(1)

    # 4️⃣ Đánh dấu các bản ghi đã bị xóa
    now_str = datetime.now().isoformat()
    for old_id, old_record in old_data_dict.items():
        if old_id not in all_rows:
            old_record["deleted_at"] = now_str
            all_rows[old_id] = old_record

    # 5️⃣ Xuất dữ liệu ra file JSON
    with open("merged_results.json", "w", encoding="utf-8") as f:
        json.dump(list(all_rows.values()), f, ensure_ascii=False, indent=4)

    print(f"✅ Dữ liệu đã được lưu vào merged_results.json với tổng {len(all_rows)} dòng!")

else:
    print("❌ Đăng nhập thất bại!", login_response.status_code, login_response.text)


✅ Đăng nhập thành công!
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 10...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 20...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 30...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 40...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 50...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 60...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 70...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 80...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 90...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 100...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 110...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 120...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 130...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 140...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 150...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 160...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 170...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 180...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 190...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 200...
🔹 Đã 

In [ ]:
import re

def refine_address(address):
    # Chuẩn hóa dấu "-" (loại bỏ khoảng trắng trước và sau)
    address = re.sub(r"\s*-\s*", "-", address)

    # Tìm số đầu tiên trong địa chỉ
    match = re.search(r"(\d+)", address)

    if match:
        house_number = int(match.group(1)) + 20  # Lấy số đầu tiên và cộng thêm 20

        # Loại bỏ phần chứa số nhà gốc, chỉ giữ lại phần đường và khu vực
        remaining_part = re.sub(r"^\D*\d+[\w/.-]*", "", address).strip(" ,.-")

        return f"{house_number} {remaining_part}"

    return address  # Giữ nguyên nếu không có số hợp lệ

# Danh sách địa chỉ cần xử lý
addresses = [
    "287 Trần Xuân Soạn",
    "135/17/23 Nguyễn Hữu Cảnh",
    "287 Trần Xuân Soạn",
    "53 Đường Số 18",
    "122A NGUYỄN THỊ THẬP",
    "42/36 Ung Văn Khiêm",
    "42/36 Ung Văn Khiêm",
    "661 quốc lộ 1a",
    "304/85/37 bùi đình túy",
    "141 Trần Thanh Mại",
    "36/32/8/7 Nguyễn Gia Trí",
    "521/96/7 Nguyễn Tri Phương",
    "76/6 ĐS 10",
    "34 VƯỜN LÀI",
    "184 LÊ ĐÌNH THÁM",
    "180 ÂU CƠ",
    "433/16 sư vạn Hạnh",
    "48/16 UNG VĂN KHIÊM",
    "5/45/2A NƠ TRANG LONG",
    "503 LÊ QUANG ĐỊNH",
    "211 QUỐC LỘ 13",
    "8 nguyễn trung trực",
    "44/1 VƯỜN LÀI",
    "1135/47 huỳnh Tấn Phát",
    "20/102 Bờ Bao Tân Thắng",
    "11/8/23 Đông Hưng Thuận 17",
    "23 nguyễn thị xiếu",
    "789 LÊ HỒNG PHONG",
    "13 Trương Phước Phan",
    "24 Đường số 2 KDC Phú Lợi",
    "288/19a Lê Văn Quới",
    "271/3B Lê Văn Lương",
    "504/58 KINH DƯƠNG VƯƠNG",
    "439 Tân Sơn",
    "3100 PHẠM THẾ HIỂN",
    "180 Gò Xoài",
    "30/100 ĐỖ NHUẬN",
    "34Bis Trần Khánh Dư",
    "3088 PHẠM THẾ HIỂN",
    "325/12 Lê Văn Quới",
    "72/10 Tây Thạnh",
    "505/1 LÊ VĂN SỸ",
    "938A Trường Chinh",
    "68 Nguyễn Thiện Thuật",
    "148/21 LÊ ĐỨC THỌ",
    "53 đường số 9",
    "196/44 Cộng Hòa, Phường 12, Tân Bình",
    "299/2 bình thành",
    "40/4 trương phước phan kp1",
    "69 TÂN THỚI NHẤT 6",
    "349/2 An Dương Vương",
    "240/2 CÁCH MẠNG THÁNG 8",
    "350/36 HUỲNH TẤN PHÁT",
    "38/55 Chế Lan Viên",
    "147/1F LÊ ĐÌNH CẨN",
    "243/22 Mã Lò",
    "107 đường số 44",
    "10 Hà Thị Khiêm",
    "625 Tên Lửa",
    "190-192 Đinh Bộ Lĩnh",
    "496/63/4A Dương Quảng Hàm",
    "148 Ung Văn Khiêm",
    "431A Lê Văn Quới",
    "53/39 Tân Chánh Hiệp 33",
    "28 VŨ HUY TẤN",
    "496/63/4A Dương Quảng Hàm",
    "3 Cộng Hoà",
    "126 Gò Dầu",
    "177 BÀ HOM",
    "128/4 Bế Văn Đàn, Bình Đường 3",
    "21/20/51 Lê Công Phép",
    "16/12 Nguyễn Văn Nghi",
    "908 Nguyễn Chí Thanh",
    "399/39 Nguyễn Đình Chiểu Q3",
    "111 Trần Trọng Cung",
    "101 Trần Trọng Cung",
    "155 Nguyễn Xí",
    "134/1 Huỳnh Văn Nghệ",
    "182/28B Lê Văn Sỹ, Phú Nhuận",
    "55/1b lý chiêu hoàng",
    "160/45/13 Nguyễn văn quỳ",
    "68 Đường số 1",
    "237/21/16 Phạm Văn Chiêu",
    "13/5 TRẦN VĂN HOÀNG",
    "30/37 Nguyễn Cửu Vân, Phường 17, Bình Thạnh",
    "54 Đường số 10",
    "23/15 Đường số 1",
    "23 đường 192",
    "80 Trần Mai Ninh",
    "C2 - C3 Hoàng Quốc Việt",
    "122/7 Yên Thế",
    "80/34 Mạc Đỉnh Chi",
    "564 Cộng Hòa",
    "62/29/11A Lâm Văn Bền",
    "51/14 Phan Huy Ích",
    "13/134 TRẦN VĂN HOÀNG",
    "606/72 Đường 3/2",
    "243/17 Tô Hiến Thành",
    "139 Đất Thánh",
    "94/9 Võ Oanh",
    "525/11A Sư Vạn Hạnh",
    "5E ĐƯỜNG SỐ 1",
    "213/4/7 đường 28, khu phố 11",
    "Số 3 -5, Đường B2, khu phố 15"
]

# Xử lý danh sách địa chỉ
refined_addresses = [refine_address(addr) for addr in addresses]

# Trả về kết quả
for addr in refined_addresses:
    print(addr)


307 Trần Xuân Soạn
155 Nguyễn Hữu Cảnh
307 Trần Xuân Soạn
73 Đường Số 18
142 NGUYỄN THỊ THẬP
62 Ung Văn Khiêm
62 Ung Văn Khiêm
681 quốc lộ 1a
324 bùi đình túy
161 Trần Thanh Mại
56 Nguyễn Gia Trí
541 Nguyễn Tri Phương
96 ĐS 10
54 VƯỜN LÀI
204 LÊ ĐÌNH THÁM
200 ÂU CƠ
453 sư vạn Hạnh
68 UNG VĂN KHIÊM
25 NƠ TRANG LONG
523 LÊ QUANG ĐỊNH
231 QUỐC LỘ 13
28 nguyễn trung trực
64 VƯỜN LÀI
1155 huỳnh Tấn Phát
40 Bờ Bao Tân Thắng
31 Đông Hưng Thuận 17
43 nguyễn thị xiếu
809 LÊ HỒNG PHONG
33 Trương Phước Phan
44 Đường số 2 KDC Phú Lợi
308 Lê Văn Quới
291 Lê Văn Lương
524 KINH DƯƠNG VƯƠNG
459 Tân Sơn
3120 PHẠM THẾ HIỂN
200 Gò Xoài
50 ĐỖ NHUẬN
54 Trần Khánh Dư
3108 PHẠM THẾ HIỂN
345 Lê Văn Quới
92 Tây Thạnh
525 LÊ VĂN SỸ
958 Trường Chinh
88 Nguyễn Thiện Thuật
168 LÊ ĐỨC THỌ
73 đường số 9
216 Cộng Hòa, Phường 12, Tân Bình
319 bình thành
60 trương phước phan kp1
89 TÂN THỚI NHẤT 6
369 An Dương Vương
260 CÁCH MẠNG THÁNG 8
370 HUỲNH TẤN PHÁT
58 Chế Lan Viên
167 LÊ ĐÌNH CẨN
263 Mã Lò
127 đường số 44
30 Hà

In [1]:
import requests
import json
import time
from datetime import datetime
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import re

# Load dữ liệu districts và wards từ file
with open("districts.json", "r", encoding="utf-8") as f:
    districts = json.load(f)

with open("wards.json", "r", encoding="utf-8") as f:
    wards_data = json.load(f)

# Map tên quận/huyện sang mã code
district_map = {d["name_with_type"]: d["code"] for d in districts}

# Map tên phường/xã sang mã code
ward_map = {}
for district_code, ward_list in wards_data.items():
    for ward in ward_list:
        ward_map[ward["name_with_type"]] = ward["code"]

# Đọc dữ liệu cũ từ merged_results.json (nếu có)
try:
    with open("merged_results.json", "r", encoding="utf-8") as f:
        old_data = json.load(f)
except FileNotFoundError:
    old_data = []

old_data_dict = {str(record.get("_id")): record for record in old_data}

session = requests.Session()

# 1️⃣ Lấy CSRF token
session.get("https://quanly.nhatrovn.vn/login")
csrf_token = session.cookies.get("XSRF-TOKEN", "")

# 2️⃣ Đăng nhập
login_response = session.post(
    "https://quanly.nhatrovn.vn/authenticate",
    headers={"X-CSRF-TOKEN": csrf_token},
    data={"_csrf": csrf_token, "j_username": "0866956654", "j_password": "12345678@X"}
)

if login_response.ok and "SUCCESS" in login_response.text:
    print("✅ Đăng nhập thành công!")

    all_rows = {}
    new_ids = []
    restored_ids = []
    last_key = 0

    def get_room_images(house_key, room_id):
        images_url = "https://quanly.nhatrovn.vn/main/room-sale/init-edit-images-room"
        try:
            response = session.post(
                images_url,
                headers={"X-CSRF-TOKEN": csrf_token},
                data={"house_key": house_key, "_id": room_id},
                timeout=5
            )
            if not response.ok:
                return []
            soup = BeautifulSoup(response.text, "html.parser")
            return [img["src"] for img in soup.find_all("img", {"class": "img-fluid"}) if "src" in img.attrs]
        except requests.exceptions.RequestException:
            return []

    def refine_address(address):
        address = re.sub(r"\s*-\s*", "-", address)
        match = re.search(r"(\d+)", address)
        if match:
            house_number = int(match.group(1)) + 20
            remaining_part = re.sub(r"^\D*\d+[\w/.-]*", "", address).strip(" ,.-")
            return f"{house_number} {remaining_part}"
        return address

    def get_fake_address(house_address):
        parts = [p.strip() for p in house_address.split(",")]
        main_address = ", ".join(parts[:-3]) if len(parts) > 3 else house_address
        return refine_address(main_address)

    while True:
        search_response = session.post(
            "https://quanly.nhatrovn.vn/main/room-sale/search",
            headers={"X-CSRF-TOKEN": csrf_token},
            data={"_lastKey": str(last_key), "sort-by": "1"}
        )

        if not search_response.ok:
            print("❌ Lỗi khi tìm kiếm:", search_response.status_code)
            break

        response_json = search_response.json()
        rows = response_json.get("rows", [])
        if not rows:
            print("✅ Không còn dữ liệu, kết thúc quá trình tìm kiếm.")
            break

        tasks = []
        with ThreadPoolExecutor(max_workers=5) as executor:
            for row in rows:
                row["_id"] = str(row.get("_id"))
                house_key = row.get("house_key", "")
                house_address = row.get("house_address", "")

                row["province_id"] = 79
                row["district_id"] = None
                row["ward_id"] = None
                row["deleted_at"] = None
                row["fake_address"] = get_fake_address(house_address)

                address_parts = house_address.split(", ")
                if len(address_parts) >= 3:
                    ward_name = address_parts[-3].strip()
                    district_name = address_parts[-2].strip()
                    row["district_id"] = district_map.get(district_name, None)
                    row["ward_id"] = ward_map.get(ward_name, None)

                tasks.append(executor.submit(get_room_images, house_key, row["_id"]))
                all_rows[row["_id"]] = row

            for future, row in zip(as_completed(tasks), rows):
                room_images = future.result()
                all_rows[row["_id"]]["images"] = room_images

                if row["_id"] not in old_data_dict:
                    new_ids.append(row["_id"])
                if row["_id"] in old_data_dict and old_data_dict[row["_id"]].get("deleted_at"):
                    row["deleted_at"] = None
                    restored_ids.append(row["_id"])

        last_key += 10
        print(f"🔹 Đã lấy {len(rows)} dòng, tiếp tục với _lastKey = {last_key}...")
        time.sleep(1)

    now_str = datetime.now().isoformat()
    for old_id, old_record in old_data_dict.items():
        if old_id not in all_rows:
            old_record["deleted_at"] = now_str
            all_rows[old_id] = old_record

    with open("merged_results.json", "w", encoding="utf-8") as f:
        json.dump(list(all_rows.values()), f, ensure_ascii=False, indent=4)

    print(f"✅ Dữ liệu đã được lưu vào merged_results.json với tổng {len(all_rows)} dòng!")

    if new_ids:
        print(f"🚀 Có {len(new_ids)} ID mới xuất hiện:")
        print("\n".join(new_ids))
    else:
        print("🔹 Không có ID mới nào.")

    if restored_ids:
        print(f"🔄 Có {len(restored_ids)} ID được khôi phục từ soft delete:")
        print("\n".join(restored_ids))
    else:
        print("🔹 Không có ID nào được khôi phục.")

else:
    print("❌ Đăng nhập thất bại!", login_response.status_code, login_response.text)


✅ Đăng nhập thành công!
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 10...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 20...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 30...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 40...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 50...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 60...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 70...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 80...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 90...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 100...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 110...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 120...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 130...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 140...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 150...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 160...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 170...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 180...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 190...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 200...


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [31]:
import requests
import json
import time
from datetime import datetime
from bs4 import BeautifulSoup
import re

# Load dữ liệu districts và wards từ file
with open("districts.json", "r", encoding="utf-8") as f:
    districts = json.load(f)

with open("wards.json", "r", encoding="utf-8") as f:
    wards_data = json.load(f)

# Map tên quận/huyện sang mã code
district_map = {d["name_with_type"]: d["code"] for d in districts}

# Map tên phường/xã sang mã code
ward_map = {}
for district_code, ward_list in wards_data.items():
    for ward in ward_list:
        ward_map[ward["name_with_type"]] = ward["code"]

# Đọc dữ liệu cũ từ merged_results.json (nếu có)
try:
    with open("merged_results.json", "r", encoding="utf-8") as f:
        old_data = json.load(f)
except FileNotFoundError:
    old_data = []

old_data_dict = {str(record.get("_id")): record for record in old_data}

session = requests.Session()

# 1️⃣ Lấy CSRF token
session.get("https://quanly.nhatrovn.vn/login")
csrf_token = session.cookies.get("XSRF-TOKEN", "")

# 2️⃣ Đăng nhập
login_response = session.post(
    "https://quanly.nhatrovn.vn/authenticate",
    headers={"X-CSRF-TOKEN": csrf_token},
    data={"_csrf": csrf_token, "j_username": "0866956654", "j_password": "12345678@X"}
)

if login_response.ok and "SUCCESS" in login_response.text:
    print("✅ Đăng nhập thành công!")

    all_rows = {}
    new_ids = []
    restored_ids = []
    last_key = 0

    def get_room_images(house_key, room_id):
        images_url = "https://quanly.nhatrovn.vn/main/room-sale/init-edit-images-room"
        try:
            response = session.post(
                images_url,
                headers={"X-CSRF-TOKEN": csrf_token},
                data={"house_key": house_key, "_id": room_id},
                timeout=5
            )
            if not response.ok:
                return []
            soup = BeautifulSoup(response.text, "html.parser")
            return [img["src"] for img in soup.find_all("img", {"class": "img-fluid"}) if "src" in img.attrs]
        except requests.exceptions.RequestException:
            return []

    def refine_address(address):
        address = re.sub(r"\s*-\s*", "-", address)
        match = re.search(r"(\d+)", address)
        if match:
            house_number = int(match.group(1)) + 20
            remaining_part = re.sub(r"^\D*\d+[\w/.-]*", "", address).strip(" ,.-")
            return f"{house_number} {remaining_part}"
        return address

    def get_fake_address(house_address):
        parts = [p.strip() for p in house_address.split(",")]
        main_address = ", ".join(parts[:-3]) if len(parts) > 3 else house_address
        return refine_address(main_address)

    while True:
        search_response = session.post(
            "https://quanly.nhatrovn.vn/main/room-sale/search",
            headers={"X-CSRF-TOKEN": csrf_token},
            data={"_lastKey": str(last_key), "sort-by": "1"}
        )

        if not search_response.ok:
            print("❌ Lỗi khi tìm kiếm:", search_response.status_code)
            break

        response_json = search_response.json()
        rows = response_json.get("rows", [])
        if not rows:
            print("✅ Không còn dữ liệu, kết thúc quá trình tìm kiếm.")
            break

        for row in rows:
            row["_id"] = str(row.get("_id"))
            house_key = row.get("house_key", "")
            house_address = row.get("house_address", "")

            row["province_id"] = 79
            row["district_id"] = None
            row["ward_id"] = None
            row["deleted_at"] = None
            row["fake_address"] = get_fake_address(house_address)

            address_parts = house_address.split(", ")
            if len(address_parts) >= 3:
                ward_name = address_parts[-3].strip()
                district_name = address_parts[-2].strip()
                row["district_id"] = district_map.get(district_name, None)
                row["ward_id"] = ward_map.get(ward_name, None)

            # Lấy ảnh từng phòng trọ tuần tự
            row["images"] = get_room_images(house_key, row["_id"])

            # Thêm vào dữ liệu tổng
            all_rows[row["_id"]] = row

            # Kiểm tra ID mới hoặc khôi phục
            if row["_id"] not in old_data_dict:
                new_ids.append(row["_id"])
            if row["_id"] in old_data_dict and old_data_dict[row["_id"]].get("deleted_at"):
                row["deleted_at"] = None
                restored_ids.append(row["_id"])

        last_key += 10
        print(f"🔹 Đã lấy {len(rows)} dòng, tiếp tục với _lastKey = {last_key}...")
        time.sleep(1)  # Nghỉ 1 giây giữa các lần request để tránh bị chặn

    now_str = datetime.now().isoformat()
    for old_id, old_record in old_data_dict.items():
        if old_id not in all_rows:
            old_record["deleted_at"] = now_str
            all_rows[old_id] = old_record

    with open("merged_results.json", "w", encoding="utf-8") as f:
        json.dump(list(all_rows.values()), f, ensure_ascii=False, indent=4)

    print(f"✅ Dữ liệu đã được lưu vào merged_results.json với tổng {len(all_rows)} dòng!")

    if new_ids:
        print(f"🚀 Có {len(new_ids)} ID mới xuất hiện:")
        print("\n".join(new_ids))
    else:
        print("🔹 Không có ID mới nào.")

    if restored_ids:
        print(f"🔄 Có {len(restored_ids)} ID được khôi phục từ soft delete:")
        print("\n".join(restored_ids))
    else:
        print("🔹 Không có ID nào được khôi phục.")

else:
    print("❌ Đăng nhập thất bại!", login_response.status_code, login_response.text)


✅ Đăng nhập thành công!
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 10...
🔹 Đã lấy 10 dòng, tiếp tục với _lastKey = 20...


KeyboardInterrupt: 